In [8]:
!pip install kaggle torch torchvision transformers matplotlib safetensors


In [9]:
from google.colab import files
files.upload()  # upload kaggle.json

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


Saving kaggle.json to kaggle.json


In [10]:
!kaggle datasets download -d jangedoo/utkface-new


Dataset URL: https://www.kaggle.com/datasets/jangedoo/utkface-new
License(s): copyright-authors
 95% 316M/331M [00:00<00:00, 417MB/s]
100% 331M/331M [00:00<00:00, 459MB/s]


In [11]:
import zipfile

with zipfile.ZipFile("utkface-new.zip", "r") as zip_ref:
    zip_ref.extractall("utkface")

print("Dataset extracted!")


Dataset extracted!


In [12]:
import os
from PIL import Image
import torch
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader


In [13]:
transform = T.Compose([
    T.Resize((224, 224)),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225])
])


In [14]:
class UTKFaceDataset(Dataset):
    def __init__(self, folder, transform=None, target="age"):
        self.folder = folder
        self.transform = transform
        self.target = target
        self.images = [f for f in os.listdir(folder) if f.endswith(".jpg")]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        path = os.path.join(self.folder, img_name)

        # filename pattern: age_gender_race_xxx.jpg
        age, gender, race = img_name.split("_")[0:3]
        age = int(age)
        gender = int(gender)
        race = int(race)

        image = Image.open(path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        if self.target == "age":
            return image, torch.tensor(age).float()
        elif self.target == "gender":
            return image, torch.tensor(gender).long()
        elif self.target == "race":
            return image, torch.tensor(race).long()


In [15]:
folder = "utkface/UTKFace"

age_dataset = UTKFaceDataset(folder, transform=transform, target="age")
gender_dataset = UTKFaceDataset(folder, transform=transform, target="gender")
race_dataset = UTKFaceDataset(folder, transform=transform, target="race")

age_loader = DataLoader(age_dataset, batch_size=32, shuffle=True)
gender_loader = DataLoader(gender_dataset, batch_size=32, shuffle=True)
race_loader = DataLoader(race_dataset, batch_size=32, shuffle=True)


In [ ]:
from transformers import ViTModel, ViTConfig
import torch.nn as nn

config = ViTConfig.from_pretrained("google/vit-base-patch16-224")
vit = ViTModel.from_pretrained("google/vit-base-patch16-224")

class AgeRegressor(nn.Module):
    def __init__(self, vit):
        super().__init__()
        self.vit = vit
        self.fc = nn.Linear(vit.config.hidden_size, 1)

    def forward(self, x):
        outputs = self.vit(x)
        x = outputs.pooler_output
        return self.fc(x)

age_model = AgeRegressor(vit)
device = "cuda" if torch.cuda.is_available() else "cpu"
age_model.to(device)


In [ ]:
import torch.optim as optim
criterion = nn.MSELoss()
optimizer = optim.AdamW(age_model.parameters(), lr=3e-5)

epochs = 10  # you can increase

for epoch in range(epochs):
    age_model.train()
    total_loss = 0

    for images, ages in age_loader:
        images = images.to(device)
        ages = ages.unsqueeze(1).to(device)

        preds = age_model(images)
        loss = criterion(preds, ages)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"AGE Epoch {epoch+1}/{epochs} - Loss: {total_loss/len(age_loader):.4f}")


In [ ]:
torch.save(age_model.state_dict(), "age_model.pt")
print("Saved age_model.pt")

from google.colab import files
files.download("age_model.pt")


In [ ]:
from transformers import ViTForImageClassification, ViTConfig

gender_config = ViTConfig.from_pretrained(
    "google/vit-base-patch16-224",
    num_labels=2  # male/female
)

gender_model = ViTForImageClassification(gender_config)
gender_model.to(device)


In [ ]:
criterion_cls = nn.CrossEntropyLoss()
optimizer = optim.AdamW(gender_model.parameters(), lr=3e-5)

epochs = 10

for epoch in range(epochs):
    gender_model.train()
    total_loss = 0

    for images, labels in gender_loader:
        images = images.to(device)
        labels = labels.to(device)

        logits = gender_model(images).logits
        loss = criterion_cls(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"GENDER Epoch {epoch+1}/{epochs} - Loss: {total_loss/len(gender_loader):.4f}")


In [ ]:
torch.save(gender_model.state_dict(), "gender_model.pt")
print("Saved gender_model.pt")

from google.colab import files
files.download("gender_model.pt")


In [ ]:
race_config = ViTConfig.from_pretrained(
    "google/vit-base-patch16-224",
    num_labels=5
)

race_model = ViTForImageClassification(race_config)
race_model.to(device)


In [ ]:
optimizer = optim.AdamW(race_model.parameters(), lr=3e-5)
epochs = 10

for epoch in range(epochs):
    race_model.train()
    total_loss = 0

    for images, labels in race_loader:
        images = images.to(device)
        labels = labels.to(device)

        logits = race_model(images).logits
        loss = criterion_cls(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"RACE Epoch {epoch+1}/{epochs} - Loss: {total_loss/len(race_loader):.4f}")


In [ ]:
torch.save(race_model.state_dict(), "race_model.pt")
print("Saved race_model.pt")

from google.colab import files
files.download("race_model.pt")
